In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [17]:
gwas_hits <- '/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/biomarkers/fine_mapping/filtered_hits/filtered.hits.99.anno.joined.tsv'

#### we use FinnGen R2 sumstats to map rsid to genomic position in GRCh38

In [36]:
finngen_r2_sumstats <- '/scratch/groups/mrivas/users/ytanigaw/20200114_FinnGen_R2/summary_stats/finngen_r2_COPD_COMORB.gz'


In [14]:
res_dir <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp_nc'
phewas_all  <- file.path(res_dir, 'ukb24983_imp_v3.glm.logistic.hybrid.gz')
phewas_hits <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.tsv')


In [3]:
phewas_df <- fread(cmd=paste('zcat', phewas_all)) %>% 
rename('CHROM' = '#CHROM')

In [4]:
phewas_df %>% select(GBE_ID) %>% unique() %>% pull() %>% length()

[1] 172

In [5]:
phewas_df %>% 
filter(
    (CHROM != 6) |
    (CHROM == 6 & POS < 25477797) | 
    (CHROM == 6 & 36448354 < POS)
) %>%
select(ID) %>% unique() %>% pull() %>% length()

[1] 2442

In [6]:
0.05 / (172 * 2442)

[1] 1.190408e-07

In [19]:
rsid_df <- fread(gwas_hits, select=c('chromosome', 'position', 'rsid'), sep='\t') %>%
rename('rsID' = 'rsid')


In [50]:
finngen_df <- fread(
    cmd=paste('zcat', finngen_r2_sumstats), 
    select=c('#chrom', 'pos', 'ref', 'alt', 'rsids')
) %>%
rename('chrom' = '#chrom') %>%
mutate(
    finngen_var_id = paste(chrom, pos, ref, alt, sep='-')
) %>%
select(rsids, finngen_var_id)


In [7]:
library(googlesheets)
gs_auth(token = "/home/users/ytanigaw/.googlesheets_token.rds")

file <- 'https://docs.google.com/spreadsheets/d/1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A'
GBE_names_df <- file %>% gs_url() %>% gs_read(ws = 'GBE_names')


Auto-refreshing stale OAuth token.
Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A
Sheet successfully identified: "GBE_names"
Accessing worksheet titled 'GBE_names'.
Parsed with column specification:
cols(
  GBE_category = col_character(),
  GBE_ID = col_character(),
  GBE_N = col_double(),
  GBE_NAME = col_character(),
  GBE_short_name = col_character(),
  GBE_short_name_len = col_double()
)


In [51]:
phewas_hits_df <- phewas_df %>% filter(
    P < 1e-7,
    (CHROM != 6) |
    (CHROM == 6 & POS < 25477797) | 
    (CHROM == 6 & 36448354 < POS)    
)  %>%
select(-REF, -ALT, -A1, -'FIRTH?', -TEST, -ERRCODE) %>%
left_join(
    GBE_names_df %>% select(GBE_ID, GBE_short_name), 
    by='GBE_ID'
) %>%
left_join(
    rsid_df, by=c('CHROM'='chromosome', 'POS'='position')
) %>%
left_join(
    finngen_df, by=c('rsID' = 'rsids')
) %>%
unique()


In [52]:
phewas_hits_df %>% dim()

[1] 268  12

In [53]:
phewas_hits_df %>% select(ID) %>% pull() %>% unique() %>% length()


[1] 175

In [54]:
phewas_hits_df %>% select(GBE_ID) %>% pull() %>% unique() %>% length()


[1] 39

In [55]:
phewas_hits_df

,CHROM,POS,ID,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,GBE_ID,GBE_short_name,rsID,finngen_var_id
,<int>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,1,22681832,1:22681832:CT:C,337146,0.890541,0.01965430,-5.89824,3.67389e-09,HC22,Osteoporosis,1:22681832_CT_C,NA
2,1,25259884,1:25259884:C:G,307048,1.044530,0.00597389,7.29272,3.03752e-13,BIN_FC5006152,Hayfever rhinitis or eczema diagnosed by doctor,rs3845302,1-24933393-C-G
3,1,27021913,1:27021913:G:C,337146,1.394200,0.05281210,6.29245,3.12496e-10,HC328,Gout,rs114165349,1-26695422-G-C
6,1,27180088,1:27180088:A:C,337146,1.375660,0.05358710,5.95173,2.65329e-09,HC328,Gout,rs75460349,1-26853597-A-C
9,1,27262545,1:27262545:C:T,337146,1.318140,0.04906670,5.62952,1.80713e-08,HC328,Gout,rs182050989,NA
11,1,109816863,1:109816863:T:TC,337146,0.892299,0.01461870,-7.79506,6.43788e-15,HC132,Angina,rs552693039,NA
14,1,151793174,1:151793174:G:T,337146,0.824196,0.03187130,-6.06648,1.30744e-09,HC201,Ulcerative colitis,rs11204894,1-151820698-G-T
15,1,151793174,1:151793174:G:T,337146,0.857512,0.02875900,-5.34513,9.03506e-08,HC95,Inflammatory bowel disease,rs11204894,1-151820698-G-T
16,1,154343714,1:154343714:CAA:C,307048,1.035930,0.00596893,5.91396,3.33978e-09,BIN_FC5006152,Hayfever rhinitis or eczema diagnosed by doctor,1:154343714_CAA_C,NA


In [56]:
phewas_hits_df %>% fwrite(phewas_hits, sep='\t')

In [57]:
phewas_hits

[1] "/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp_nc/ukb24983_imp_v3.nc.phewas.hits.tsv"